**0. Melakukan autentikasi pada Google**

In [1]:
# import library untuk meload file dari google drive
!pip install -U -q PyDrive

# import untuk keperluan auth
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# melakukan auth
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [2]:
# melakukan mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**1. Membaca dataset yang berbentuk CSV**

In [3]:
# import library untuk membaca dataset
import numpy as np
import pandas as pd

# membaca dataset dari file google drive
path = "/content/drive/MyDrive/AI_2021/datasetv1.csv"
df = pd.read_csv(path, sep=',', encoding='utf-8')

# menampilkan dataset
df.head()

,Id,sentiment,text
0,1,negative,<USERNAME> TOLOL!! Gak ada hubungan nya kegug...
1,2,negative,Geblek lo tata...cowo bgt dibela2in balikan......
2,3,negative,Kmrn termewek2 skr lengket lg duhhh kok labil ...
3,4,negative,"Intinya kalau kesel dengan ATT nya, gausah ke ..."
4,5,negative,"hadewwwww permpuan itu lg!!!!sakit jiwa,knp ha..."


**2. Membersihkan data : Casefolding**

Tahapan pertama dari preprocessing data dan casefolding:
1. Menghilangkan tanda baca, emoticon, hashtag, atau tagging username
2. Mengubah semua kata menjadi lowercase (casefolding), jadi ketika ada kata yang sama namun memiliki susunan kapital yang berbeda, maka kata tersebut akan diseragamkan menjadi kata yang sama.

In [4]:
# import library regex
import re

# casefolding pada dataset
def dataset_casefolding(stc):
  stc = re.sub("<USERNAME>", "", stc)
  stc = stc.lower()
  stc = stc.strip(" ")
  stc = re.sub(r'[?|$|.|2!_:")(-+,]', '', stc)
  
  return stc

# mengapply pada dataset
df['text'] = df['text'].apply(dataset_casefolding)

# menampilkan dataset
df.head(10)

,Id,sentiment,text
0,1,negative,tolol gak ada hubungan nya keguguran dgn pake ...
1,2,negative,geblek lo tatacowo bgt dibelain balikanhadewwn...
2,3,negative,kmrn termewek skr lengket lg duhhh kok labil b...
3,4,negative,intinya kalau kesel dengan att nya gausah ke a...
4,5,negative,hadewwwww permpuan itu lgsakit jiwaknp harus d...
5,6,negative,pantesan di tinggalin laki ya lakinya juga mik...
6,7,negative,kebiasaan balajaer nyampah d ig para artissuka...
7,8,negative,krn sebagian besar rakyat indonesia itu bodoh ...
8,9,negative,ayu janda bego pny suami kpn nikah laginya
9,10,negative,anyiennnnggg suaranya ancur banget lebih merdu...


**3. Melakukan tokenizing pada dataset**

Yaitu memecah kalimat yang perada pada *panda series* menjadi kata-kata yang unik (tidak berulang)


Misal kata-kata = ['aku akan pergi namun aku tidak akan pulang']

menjadi = [aku, akan, pergi, namun, tidak, pulang]

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np

# proses tokenizing
def tokenizing(stc):
  stc = [stc]
  token = Tokenizer()         # buat instance tokenizer
  token.fit_on_texts(stc)     # load text di instance nya

  return list(token.index_word.values())

df['text'] = df['text'].apply(tokenizing)
df.head()

,Id,sentiment,text
0,1,negative,"[gak, nya, ada, hubungan, lo, tolol, keguguran..."
1,2,negative,"[geblek, lo, tatacowo, bgt, dibelain, balikanh..."
2,3,negative,"[kmrn, termewek, skr, lengket, lg, duhhh, kok,..."
3,4,negative,"[dia, sama, anaknya, anak, orang, bener, benci..."
4,5,negative,"[hadewwwww, permpuan, itu, lgsakit, jiwaknp, h..."


**3. Proses menghapus stopword**

Stopword ialah ata umum yang tidak memberikan informasi penting (yang biasanya tidak diacuhkan atau dibuang (yang, di, ke)). Dalam tahapan ini, kata-kata yang termasuk dalam stopword akan dihilangkan. 

In [6]:
# process filtering stopword (Kata umum yang tidak memberikan informasi penting (yang biasanya tidak diacuhkan atau dibuang, misalnya dalam proses pembuatan indeks))
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

def stopwords_removal(stc):
  filtering = stopwords.words('indonesian', 'english')
  filtering.append('yg')
  x = []
  data = []

  def if_nin(x):
    if x in filtering:
      return False

    return True

  fit = filter(if_nin, stc)
  for x in fit:
    data.append(x)

  return data

df['text'] = df['text'].apply(stopwords_removal)
df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,Id,sentiment,text
0,1,negative,"[gak, nya, hubungan, lo, tolol, keguguran, dgn..."
1,2,negative,"[geblek, lo, tatacowo, bgt, dibelain, balikanh..."
2,3,negative,"[kmrn, termewek, skr, lengket, lg, duhhh, labi..."
3,4,negative,"[anaknya, anak, orang, bener, benci, intinya, ..."
4,5,negative,"[hadewwwww, permpuan, lgsakit, jiwaknp, jd, pe..."


**4. Proses stemming**

Yaitu proses dimana kata-kata yang berada di dalam kalimat pada dataset, akan diubah menjadi kata dasarnya. 

Contoh = hubungan menjadi hubung

In [7]:
# proses stemming
! pip install Sastrawi

from sklearn.pipeline import Pipeline
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

def stemming(stc):
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()
  do = []

  for word in stc:
    dt = stemmer.stem(word)
    do.append(dt)

  d_clean = []
  d_clean = " ".join(do)

  return d_clean
  
df['text'] = df['text'].apply(stemming)
df.head()

df.to_csv('data_clean.csv', index=False)
data_clean = pd.read_csv('data_clean.csv', encoding='latin1')
data_clean.head()

     |████████████████████████████████| 215kB 5.6MB/s 


,Id,sentiment,text
0,1,negative,gak nya hubung lo tolol gugur dgn pake hijab s...
1,2,negative,geblek lo tatacowo bgt bain balikanhadewwntar ...
2,3,negative,kmrn mewek skr lengket lg duhhh labil bgt sih ...
3,4,negative,anak anak orang bener benci inti kesel att nya...
4,5,negative,hadewwwww permpuan lgsakit jiwaknp jd peran ut...


In [8]:
data_clean = data_clean.astype({'sentiment': 'category'})
data_clean = data_clean.astype({'text': 'string'})

**6. Proses Pembobotan Kata**

Menggunakan TfidfVectorizer, untuk memboboti suatu kata. Semakin banyak kata yang terdapat pada kalimat, maka bobot akan semakin besar

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer()
text_tf = tf.fit_transform(data_clean['text'].astype('U'))
text_tf

<400x2602 sparse matrix of type '<class 'numpy.float64'>'
	with 6133 stored elements in Compressed Sparse Row format>

**7. Memisah data, menjadi training dan test data dengan proporsi 80:20**

In [10]:
# splitting data

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(text_tf, data_clean['sentiment'], test_size=0.2, random_state=42)

**8. Menjalankan model naive bayes**

- Membuat instance model naive bayes, lalu memasukkan data traininnya (sebesar 80 persen dari keseluruhan data).

- Kemudian setelah di training, akan dilakukan prengujian (test) pada model, dengan memasukkan data testingnya

- Setelah hasil testing diperoleh, kemudian akan dihitungkan keakuratannya.

In [11]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
clf = MultinomialNB().fit(X_train, y_train)
predicted = clf.predict(X_test)

print("MultinomialNB accurancy : ", accuracy_score(y_test, predicted))

MultinomialNB accurancy :  0.925


**9. Testing model**

- Secara manual
- Dengan csv

In [12]:
# Testing input manual
inpx = input('Masukkan kalimat\t:\t')
in_pds = pd.Series(np.array([inpx]))
in_tf = tf.transform(in_pds)

clf.predict(in_tf)

Masukkan kalimat	:	sampah kali negri ini!


array(['negative'], dtype='<U8')

In [13]:
# Testing dengan csv

# membaca dataset dari file google drive
import collections

path = "/content/drive/MyDrive/AI_2021/trydata.csv"
df2 = pd.read_csv(path, sep=',', encoding='utf-8')
df2_raw = pd.read_csv(path, sep=',', encoding='utf-8')['values']

# casefolding
df2['values'] = df2['values'].apply(dataset_casefolding)
# tokenizing
df2['values'] = df2['values'].apply(tokenizing)
# menghilangkan stopwords
df2['values'] = df2['values'].apply(stopwords_removal)
# stemming
df2['values'] = df2['values'].apply(stemming)
df2.to_csv('data_clean2.csv', index=False)
data_clean2 = pd.read_csv('data_clean2.csv', encoding='latin1')
# pembobotan kata
text_tf2 = tf.transform(data_clean2['values'].astype('U'))

# prediksi melalui model
predictions = clf.predict(text_tf2)
tweets_len = len(predictions)

# hitung presentase
tweets_p = np.count_nonzero(predictions == 'positive')
tweets_n = np.count_nonzero(predictions == 'negative')
c = (tweets_p/tweets_len) * 100
prec_p = (tweets_p/tweets_len) * 100
prec_n = (tweets_n/tweets_len) * 100

print('Berdasarkan username Twitter yang Anda berikan, kami telah menganalisis kemungkinan kondisi emosional anak Anda beberapa waktu belakangan ini, memiliki presentase sebagai berikut:\n')
print('positive : ', prec_p)
print('negative : ', prec_n, '\n')
if prec_p > prec_n:
  print('Maka dari itu, sejauh ini keadaan emosional anak Bapak/Ibu berada di kondisi mental yang positif.')
elif prec_p < prec_n:
  print('Maka dari itu kami sarankan Bapak/Ibu agar lebih meluangkan waktunya bersama anak Anda supaya lebih mengerti kondisi emosionalnya.')
else:
  print('Sejauh ini tweet yang diposting oleh anak Anda memiliki memiliki presentasi yang setara, kami menyarankan agar Anda lebih memperhatikan anak anda agar kondisi emosional anak Anda kembali stabil')

Berdasarkan username Twitter yang Anda berikan, kami telah menganalisis kemungkinan kondisi emosional anak Anda beberapa waktu belakangan ini, memiliki presentase sebagai berikut:

positive :  20.0
negative :  80.0 

Maka dari itu kami sarankan Bapak/Ibu agar lebih meluangkan waktunya bersama anak Anda supaya lebih mengerti kondisi emosionalnya.
